In [1]:
import ee

In [2]:
ee.Initialize()



In [76]:
'''###########################################/
   This script was created by SkyTruth to identify possible active mountaintop removal 
   and other surface coal mining.
   
   Playground code: https:#code.earthengine.google.com/02cbf97172a6dd9d1062ca0c6c122137
###########################################'''

'''------------------------------------ IMPORT STUDY AREA ----------------------------'''
campagna_study_area = ee.FeatureCollection('ft:1Qo6AmhdEN44vPUpyGtzPtQUUO4rygWv4MljZ-MiE');
# Get the link here: https:#www.google.com/fusiontables/DataSource?docid=1Qo6AmhdEN44vPUpyGtzPtQUUO4rygWv4MljZ-MiE

'''------------------------------------ SET NDVI THRESHOLD ---------------------------'''
NDVI_Threshold = 0.51;

'''------------------------------------- IMPORT MASK ---------------------------------'''
mask_input_60m_2015 = ee.Image('users/jerrilyn/2015mask-PM-fullstudy-area');
# Get the link here: https:#drive.google.com/file/d/0B_MArPTqurHudFp6STU4ZzJHRmc/view

''' ----------------------------------- VISUALIZATION / SETUP ------------------------------
Adds Campagna study area; various areas of interest  '''

# Map.addLayer(campagna_study_area, {}, "Study area");

# Choose your favorite area of interest! Comment out all but one:
#Map.centerObject(campagna_study_area);        # Full study extent
#Map.setCenter(-81.971744, 38.094253, 12);     # Near Spurlockville, WV
# Map.setCenter(-82.705444, 37.020257, 12);     # Near Addington, VA
#Map.setCenter(-83.224567, 37.355144, 11);     # Near Dice, KY
#Map.setCenter(-83.931184, 36.533646, 12);     # Near Log Mountain, TN

# This list will contain all output images, so as to build an ImageCollection later for video export
allMTR_list = [];

# This empty 2D array will be used for area calculation later (if performed)
MTR_area = [[],[]];

# Subregions for exporting images/videos
# geometryI   = ee.Geometry.Rectangle([-79.612, 37.3525, -82.421, 39.037])#.toGeoJSON();
# geometryII  = ee.Geometry.Rectangle([-82.421, 37.3525, -84.993, 38.942]).toGeoJSON();
# geometryIII = ee.Geometry.Rectangle([-82.421, 35.367,  -85.811, 37.3525]).toGeoJSON();
# geometryIV  = ee.Geometry.Rectangle([-79.849, 35.763,  -82.421, 37.3525]).toGeoJSON();
# exportbounds = campagna_study_area.geometry().bounds().getInfo();

counties = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM');
boon = counties.filterMetadata('FIPS', 'equals', 54005);

'''--------------------------------- IMAGE PROCESSING ---------------------------------'''
for year in range(1984,2016): # Years of interest for the study
  

    # Determine what imagery dataset to use, based off year loop; and what threshold to use
    if year <= 2011:
        imagery = ee.ImageCollection("LANDSAT/LT5_L1T_ANNUAL_GREENEST_TOA");
        NDVIbands = 43;
    
    elif year == 2012:
        imagery = ee.ImageCollection("LANDSAT/LE7_L1T_ANNUAL_GREENEST_TOA");
        NDVIbands = 43;
    
    elif year >= 2013:
        imagery = ee.ImageCollection("LANDSAT/LC8_L1T_ANNUAL_GREENEST_TOA");
        NDVIbands = 54; # Because different bands are needed for LS8

    # Select specific year for analysis
    yearImg = ee.Image(imagery.filterDate(str(year)+"-01-01", str(year)+"-12-31").first()).clip(campagna_study_area);

    # Calculate NDVI (using normalizedDifference function; select correct bands per sensor)
    if NDVIbands == 54:
        NDVI = yearImg.normalizedDifference(["B5","B4"]).clip(campagna_study_area);
    else:
        NDVI = yearImg.normalizedDifference(["B4","B3"]).clip(campagna_study_area);
    

    # Create a mask of areas that ARE NOT mines (value of 1 to locations where NDVI is <= threshold)
    lowNDVI = NDVI.where(NDVI.lte(NDVI_Threshold),1).where(NDVI.gt(NDVI_Threshold),0);

    # Create binary image containing the intersection between the LowNDVI and anywhere the inverted mask is 1
    MTR = lowNDVI.And(mask_input_60m_2015.eq(0));

    # Erode/dilate MTR sites to remove outliers (pixel clean-up)
    MTR_eroded_dialated_dialated_eroded = MTR.reduceNeighborhood(ee.Reducer.max(), ee.Kernel.euclidean(30, 'meters'))\
    .reduceNeighborhood(ee.Reducer.min(), ee.Kernel.euclidean(30, 'meters'))\
    .reduceNeighborhood(ee.Reducer.min(), ee.Kernel.euclidean(30, 'meters'))\
    .reduceNeighborhood(ee.Reducer.max(), ee.Kernel.euclidean(30, 'meters'));

    MTR_masked = MTR_eroded_dialated_dialated_eroded.updateMask(MTR_eroded_dialated_dialated_eroded);

    # Get a pixel area image, which will apply to any scale you provide
    Area = ee.Image.pixelArea();

    # The area calculation, which currently burns out the server
    areaAll = MTR_masked.multiply(Area).reduceRegion(reducer=ee.Reducer.sum(),geometry=boon, 
                                                      scale= 30,crs= 'EPSG:3857',maxPixels= 1e9)
    areaKmSq = ee.Number(areaAll.get('constant')).divide(1000*1000);
    

    # Add these areas and their corresponding year to 2D array
    MTR_area[0].append(year);
    MTR_area[1].append(areaKmSq);
    allMTR_list.append(MTR_masked);



In [47]:
dir(MTR_area[1][0])

['And',
 'Not',
 'Or',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__eq__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cast',
 '_initialized',
 '_number',
 'abs',
 'acos',
 'add',
 'args',
 'aside',
 'asin',
 'atan',
 'atan2',
 'bitCount',
 'bitwiseAnd',
 'bitwiseNot',
 'bitwiseOr',
 'bitwiseXor',
 'bitwise_and',
 'bitwise_not',
 'bitwise_or',
 'bitwise_xor',
 'byte',
 'cbrt',
 'ceil',
 'cos',
 'cosh',
 'digamma',
 'divide',
 'double',
 'encode',
 'eq',
 'erf',
 'erfInv',
 'erfc',
 'erfcInv',
 'exp',
 'first',
 'firstNonZero',
 'first_nonzero',
 'float',
 'floor',
 'format',
 'freeze',
 'func',
 'gamma',
 'getInfo',
 'gt',
 'gte',
 'hypot',
 'initialize',
 'int',
 'int16',
 'int32',
 'int64',
 'int8',
 'isVariable',
 'lanczos',
 'leftShift',
 'left_shift',
 'log',
 'log10',
 'long',
 'lt',


In [117]:
print(MTR_area[1][0].getInfo())

EEException: Dictionary.get: Dictionary does not contain key: constant.